In [0]:
# import libs 

import tensorflow as tf # for deep learning 

# helper library 
import numpy as np # for matrix maths 

In [2]:
# importing dataset 
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('./data/MNIST/', one_hot=True)

# train_df = pd.read_csv('./fashion-mnist_train.csv')

# train_df.describe()

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/MNIST/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/MNIST/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./data/MNIST/t10k-images-idx3-ubyte.gz
Extracting ./data/MNIST/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [0]:
# helper function for conv_layer 
def conv_layer(input, size_in, size_out, name="conv"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([5,5, size_in, size_out], stddev=0.1), name="W")
        b = tf.Variable(tf.truncated_normal([size_out], stddev=0.1), name="B")
        
        conv = tf.nn.conv2d(input, w, strides=[1,1,1,1], padding="SAME")
        act = tf.nn.relu(conv + b)
        
        # writing summary 
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activation", act)
        
        # return a pooled version 
        return tf.nn.max_pool(act, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

In [0]:
# helper function for fully connected layer 
def fc_layer(input, size_in, size_out, name="fc"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([size_in, size_out], stddev=0.1), name="W")
        b = tf.Variable(tf.truncated_normal([size_out], stddev=0.1), name="B")
        
        # activation 
        act = tf.matmul(input, w) + b
        
        # writing summaries 
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activation", act)
        
        return act

In [5]:
# model arctitecture 

tf.reset_default_graph() # reset all previous graphs
sess = tf.Session() # start a session

# a placeholder for the images and labels 
x = tf.placeholder(tf.float32, shape=[None, 784])
x_image = tf.reshape(x, shape=[-1, 28, 28, 1])

# summary for the x_image
tf.summary.image("input", x_image, 3)

y = tf.placeholder(tf.int32, shape=[None,10])

# wiring model
# conv layers 
conv1 = conv_layer(x_image, 1, 32, "conv1")
conv2 = conv_layer(conv1, 32, 64, "conv2")

# flattened layer 
flattened = tf.reshape(conv2, [-1, 7 * 7 * 64])

# making a fully connected layer 
fc1 = fc_layer(flattened, 7 * 7 * 64, 1024, name="fc1")
relu = tf.nn.relu(fc1)

# summary for fc layer 1 
tf.summary.histogram("fc1/relu", relu)
logits = fc_layer(relu,1024, 10, name="fc2")

# some important params for the model eval and train 

with tf.name_scope("xent"):
    xent = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(
            logits=logits, labels=y), name="xent")
    tf.summary.scalar("xent", xent)

with tf.name_scope("train"):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(xent)

with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar("accuracy", accuracy)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [0]:
# merging all the summaries and saving a session graph 

summ = tf.summary.merge_all()

# variables initializer 
sess.run(tf.global_variables_initializer())

# a file writer for saving summaries 
writer = tf.summary.FileWriter('./logs')

# write session graph
writer.add_graph(sess.graph)

In [8]:
# training the model 
for i in range(2001):
    batch = mnist.train.next_batch(100)
    if i % 5 == 0:
        [train_accuracy, s] = sess.run([accuracy, summ], feed_dict={x:batch[0], y:batch[1]})
    writer.add_summary(s, i)
    if i % 10 == 0:
      print("accuracy {} at step {}".format(train_accuracy, i))
    
    sess.run(train_step, feed_dict={x:batch[0], y:batch[1]})

accuracy 0.8999999761581421 at step 0
accuracy 0.949999988079071 at step 10
accuracy 0.9200000166893005 at step 20
accuracy 0.9100000262260437 at step 30
accuracy 0.9599999785423279 at step 40
accuracy 0.949999988079071 at step 50
accuracy 0.9200000166893005 at step 60
accuracy 0.8999999761581421 at step 70
accuracy 0.949999988079071 at step 80
accuracy 0.8999999761581421 at step 90
accuracy 0.9599999785423279 at step 100
accuracy 0.9300000071525574 at step 110
accuracy 0.9399999976158142 at step 120
accuracy 0.9800000190734863 at step 130
accuracy 0.9599999785423279 at step 140
accuracy 0.9700000286102295 at step 150
accuracy 0.9700000286102295 at step 160
accuracy 0.9599999785423279 at step 170
accuracy 0.9800000190734863 at step 180
accuracy 0.9599999785423279 at step 190
accuracy 0.949999988079071 at step 200
accuracy 0.9900000095367432 at step 210
accuracy 0.9599999785423279 at step 220
accuracy 0.9399999976158142 at step 230
accuracy 0.9599999785423279 at step 240
accuracy 0.9800